In [12]:
from playsound import playsound
import warnings 
warnings.filterwarnings('ignore')

In [13]:
file = "output_t_0.wav"
playsound(file)


    Error 263 for command:
        close output_t_0.wav
    The specified device is not open or is not recognized by MCI.
Failed to close the file: output_t_0.wav


In [14]:
import keras
import numpy as np
import librosa

class livePredictions:
    def __init__(self, path, file):
        self.path = path
        self.file = file

    def load_model(self):
        self.loaded_model = keras.models.load_model(self.path)

    def makepredictions(self):
        data, sampling_rate = librosa.load(self.file)
        mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=40).T, axis=0)
        x = np.expand_dims(mfccs, axis=1)
        x = np.expand_dims(x, axis=0)
        predictions = np.argmax(self.loaded_model.predict(x),axis=1)
        return "Emotion is "+self.convertclasstoemotion(predictions)

    @staticmethod
    def convertclasstoemotion(pred):
        label_conversion = {'0': 'neutral',
                            '1': 'calm',
                            '2': 'happy',
                            '3': 'sad',
                            '4': 'angry',
                            '5': 'fearful',
                            '6': 'disgust',
                            '7': 'surprised'}
        for key, value in label_conversion.items():
            if int(key) == pred:
                label = value
        return label
pred = livePredictions(path='testing_model(ADAM).h5',file="output_t_0.wav")
pred.load_model()
pred.makepredictions()

'Emotion is calm'

In [4]:
import sqlite3
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
conn = sqlite3.connect('Emotion Classifier.sqlite')
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS SER_data (emotion VARCHAR,recorded_time VARCHAR,file VARCHAR)')
cur.execute('INSERT INTO SER_data (emotion,recorded_time,file) VALUES (?, ?, ?)',
            (pred.makepredictions(), current_time, file))
conn.commit()
cur.execute('SELECT * FROM SER_data')
data = cur.fetchall()
print(data)

[('Prediction is fearful', '22:38:39', 'Ravtess\\03-01-06-01-02-02-16.wav'), ('Prediction is fearful', '11:22:30', 'Ravtess\\03-01-06-01-02-02-16.wav')]


In [5]:
dropTableStatement = "DROP TABLE SER_data"
print(f"Table Data:{data}")
cur.execute(dropTableStatement)
print("Table Deleted!")

Table Data:[('Prediction is fearful', '22:35:20', 'Ravtess\\03-01-06-01-02-02-16.wav')]
Table Deleted!


In [17]:
import csv
import os
cur.execute("select * from SER_data")
with open("ser_data.csv", "w") as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=" ")
    csv_writer.writerow([i[0] for i in cur.description])
    csv_writer.writerows(cur)

dirpath = os.getcwd() + "/ser_data.csv"
print("Data exported Successfully into {}".format(dirpath))

Data exported Successfully into C:\Users\astro/ser_data.csv


In [18]:
import pandas as pd
data = pd.read_csv('ser_data.csv')
data

,emotion recorded_time file
0,Prediction is fearful 22:38:39 Ravtess\03-01-0...
1,Prediction is fearful 11:22:30 Ravtess\03-01-0...


In [19]:
import csv
from collections import defaultdict

columns = defaultdict(list)
with open('ser_data.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    headers = next(reader)
    column_nums = range(len(headers)) # Do NOT change to xrange
    for row in reader:
        for i in column_nums:
            columns[headers[i]].append(row[i])
# Following line is only necessary if you want a key error for invalid column names
columns = dict(columns)

IndexError: list index out of range